In [0]:
import pyspark.sql.functions as F
import pyspark.sql.types as T

import logging
import json 


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
DISPLAY_LIMIT = 20

### The structure
1. Read data from zipped json files into one dataframe.
2. Apply necessary (cleaning) transformations to the dataframe.
3. Create the new DataFrames corresponding to our Warehouse Schema.  
4. Save the DataFrames as Delta tables.
5. Demonstrate adding new entries to the warehouse.
6. Demonstrate queries on the data.

### 1. Extract the data

In [0]:
# Uncomment one or the other line.

# This reads ALL splits into one dataframe
#_df = spark.read.option("multiline", True).json('dbfs:/user/dblpv13/dblpv13.*.json.gz')

# For testing, read just one split
_df = spark.read.option("multiline", True).json('dbfs:/user/dblpv13/dblpv13.0.json.gz')

In [0]:
# Immediately delete the abstract column because they look really annoying on GitHub.
_df = _df.drop(F.col('abstract'))

_df.printSchema()

root
 |-- _id: string (nullable = true)
 |-- authors: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _id: string (nullable = true)
 |    |    |-- bio: string (nullable = true)
 |    |    |-- email: string (nullable = true)
 |    |    |-- gid: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- name_zh: string (nullable = true)
 |    |    |-- oid: string (nullable = true)
 |    |    |-- oid_zh: string (nullable = true)
 |    |    |-- orcid: string (nullable = true)
 |    |    |-- org: string (nullable = true)
 |    |    |-- org_zh: string (nullable = true)
 |    |    |-- orgid: string (nullable = true)
 |    |    |-- orgs: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- orgs_zh: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- sid: string (nullable = true)
 |-- doi: string (nullable = true)
 |-- fos: array (nullable = 

### 2. Transform the data
TODO: delete all authors where author id is null.

TODO: delete all entries where org id is null.

TODO: delete all entries where venue id is null.

We don't want any null values in the FK columns.

In [0]:
logger.info(f"Initially, there were {_df.count()} rows of data")

INFO:__main__:Initially, there were 250000 rows of data


In [0]:
# Drop entries with 1-word titles or empty authors or nonexistant _id or 
_df = (_df.filter((F.size(F.col('authors')) > 0) & # By default F.size() returns -1 if the value is null.
                  (F.size(F.split(F.col('title'), ' ')) > 1) &  
                  (F.col('_id') != '') & 
                  (F.col('_id').isNotNull()) & 
                  ~(F.array_contains(F.col('references'), ''))))

In [0]:
# remove forewords
_df = _df.filter(~F.lower(F.col("title")).contains("foreword"))

In [0]:
# Convert n_citation data type to int
_df = _df.withColumn('n_citation', F.col('n_citation').cast('int'))

In [0]:
# Replace empty language values with null.
_df = _df.withColumn('lang', F.when(F.col('lang') == '', None).otherwise(F.col('lang')))

In [0]:
# Replace empty 'keyword' and 'fos' arrays with null values.
_df = (_df.withColumn('keywords', F.when(F.size(F.col('keywords')) == 0, None).otherwise(F.col('keywords')))
          .withColumn('fos', F.when(F.size(F.col('fos')) == 0, None).otherwise(F.col('fos'))))

In [0]:
# Replace non-numeric page numbers with nulls and convert column type to int. Then replace 0 page numbers with nulls as well.
_df = (_df.withColumn('page_start', F.when(F.col('page_start').cast('int').isNotNull(), F.col('page_start')).otherwise(None)) # replace non-numeric page numbers with null
          .withColumn('page_end', F.when(F.col('page_end').cast('int').isNotNull(), F.col('page_end')).otherwise(None))
          .withColumn('page_start', F.col('page_start').cast('int')) # convert column type to int
          .withColumn('page_end', F.col('page_end').cast('int'))
          .withColumn('page_start', F.when(F.col('page_start') == 0, None).otherwise(F.col('page_start'))) # replace 0 page numbers with null as well
          .withColumn('page_end', F.when(F.col('page_end') == 0, None).otherwise(F.col('page_end'))))

In [0]:
# Replace empty dois with nulls.
_df = _df.withColumn('doi', F.when(F.col('doi') == '', None).otherwise(F.col('doi')))

In [0]:
# Replace empty years with nulls and change data type to int.
_df = (_df.withColumn('year', F.when(F.col('year') == 0, None).otherwise(F.col('year')))
          .withColumn('year', F.col('year').cast('int')))

In [0]:
# Replace non-numeric volume and issue numbers with null and convert data types to int. Then repalce 0 values with null as well.
_df = (_df.withColumn('volume', F.when(F.col('volume').cast('int').isNotNull(), F.col('volume')).otherwise(None)) # replace non-numeric values
          .withColumn('issue', F.when(F.col('issue').cast('int').isNotNull(), F.col('issue')).otherwise(None))
          .withColumn('volume', F.col('volume').cast('int')) # convert column type to int
          .withColumn('issue', F.col('issue').cast('int'))
          .withColumn('volume', F.when(F.col('volume') == 0, None).otherwise(F.col('volume'))) # replace 0 issue and volume numbers with null as well.
          .withColumn('issue', F.when(F.col('issue') == 0, None).otherwise(F.col('issue'))))

In [0]:
# replace empty strings in some columns with nulls
def replace_empty_string(col):
    return F.when(col == "", None).otherwise(col)

venue = F.col("venue")
for col in ["_id", "issn", "name", "name_d", "name_s", "online_issn", "publisher", "raw", "raw_zh", "t"]:
    venue = venue.withField(col, replace_empty_string(F.col(f"venue.{col}")))
    
_df = (
    _df
    .withColumn("venue", venue)
    .withColumn("issn", replace_empty_string(F.col("issn")))
    .withColumn("isbn", replace_empty_string(F.col("isbn")))
    .withColumn("isbn", F.when(F.col("isbn") == "isbn", None).otherwise(F.col("isbn")))
    .withColumn("issn", F.when(F.col("issn") == "issn", None).otherwise(F.col("issn")))
)

In [0]:
# fix incorrect issn
_df = (_df
           .withColumn("issn",
                       F.when(F.length(F.col("issn")) == 9, F.col("issn"))
                       .when(F.length(F.col("issn")) == 8, F.concat(F.col("issn").substr(1, 4), F.lit("-"), F.col("issn").substr(5, 4)))
                       .when(F.col("issn").contains("E-ISBN"), F.col("issn").substr(1, 9))
                       .otherwise(None)
                      )
           .withColumn("venue", 
                       F.col("venue")
                       .withField("issn", F.coalesce(F.col("venue.issn"), F.col("issn")))
                       )
           .drop("issn")
          )

In [0]:
# replace venue with null fields with null
venue_is_empty = (
    F.col("venue.issn").isNull() &
    F.col("venue.name").isNull() &
    F.col("venue.name_d").isNull() &
    F.col("venue.name_s").isNull() &
    F.col("venue.online_issn").isNull() &
    F.col("venue.publisher").isNull() &
    F.col("venue.raw").isNull() &
    F.col("venue.raw_zh").isNull()
)
_df = _df.withColumn("venue", F.when(venue_is_empty, None).otherwise(F.col("venue")))

In [0]:
logger.info(f"Now, there are {_df.count()} rows of data")

_df.printSchema()

INFO:__main__:Now, there are 242656 rows of data
root
 |-- _id: string (nullable = true)
 |-- authors: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _id: string (nullable = true)
 |    |    |-- bio: string (nullable = true)
 |    |    |-- email: string (nullable = true)
 |    |    |-- gid: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- name_zh: string (nullable = true)
 |    |    |-- oid: string (nullable = true)
 |    |    |-- oid_zh: string (nullable = true)
 |    |    |-- orcid: string (nullable = true)
 |    |    |-- org: string (nullable = true)
 |    |    |-- org_zh: string (nullable = true)
 |    |    |-- orgid: string (nullable = true)
 |    |    |-- orgs: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- orgs_zh: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- sid: string (nullable = true)
 |-- doi: stri

In [0]:
# Display endresult
display(_df.limit(DISPLAY_LIMIT))

_id,authors,doi,fos,isbn,issue,keywords,lang,n_citation,page_end,page_start,pdf,references,title,url,venue,volume,year
53e99784b7602d9701f3e133,"List(List(53f45728dabfaec09f209538, null, null, null, Peijuan Wang, null, null, null, null, null, null, null, null, null, null), List(5601754345cedb3395e59457, null, null, null, Jiahua Zhang, null, null, null, null, null, null, null, null, null, null), List(53f38438dabfae4b34a08928, null, null, null, Donghui Xie, null, null, null, null, null, null, null, null, null, null), List(5601754345cedb3395e5945a, null, null, null, Yanyan Xu, null, null, null, null, null, null, null, null, null, null), List(53f43d25dabfaeecd6995149, null, null, null, Yun Xu, null, null, null, null, null, null, null, null, null, null))",10.1109/IGARSS.2011.6049503,"List(Agronomy, Moisture, Hydrology, Environmental science, Dry weight, Water content, Stomatal conductance, Transpiration, Irrigation, Soil water, Canopy)",null,null,"List(canopy parameters, canopy spectrum, different soil water content control, winter wheat, irrigation, hydrology, radiometry, moisture, indexes, vegetation, indexation, dry weight, soil moisture, water content, indexing terms, spectrum, natural disaster)",en,0,1933,1930,null,null,The relationship between canopy parameters and spectrum of winter wheat under different irrigations in Hebei Province.,List(http://dx.doi.org/10.1109/IGARSS.2011.6049503),"List(53a7297d20f7420be8bd4ae7, null, null, International Geoscience and Remote Sensing Symposium, null, null, null, IGARSS, null, null, null, null, 0)",null,2011
53e99784b7602d9701f3e151,"List(List(53f46797dabfaeb22f542630, null, null, null, Jairo Rocha, null, null, null, null, null, null, null, null, null, null), List(54328883dabfaeb4c6a8a699, null, null, null, Theo Pavlidis, null, null, null, null, null, null, null, null, null, null))",10.1109/ICDAR.1993.395663,"List(Intelligent character recognition, Pattern recognition, Computer science, Feature (computer vision), Document processing, Handwriting recognition, Optical character recognition, Feature extraction, Feature (machine learning), Artificial intelligence, Intelligent word recognition)",null,null,"List(handwriting recognition, prototypes, image segmentation, computer science, expert systems, knowledge base, pattern recognition, usability, optical character recognition, shape, feature extraction)",en,17,605,602,null,"List(53e99cf5b7602d97025ace63, 557e8a7a6fee0fe990caa63d, 53e9a96cb7602d97032c459a, 53e9b929b7602d9704515791, 557e59ebf6678c77ea222447)",A solution to the problem of touching and broken characters.,List(http://dx.doi.org/10.1109/ICDAR.1993.395663),"List(53a72a4920f7420be8bfa51b, null, null, International Conference on Document Analysis and Recognition, null, null, null, ICDAR-1, null, null, null, null, 0)",null,1993
53e99784b7602d9701f3e15d,"List(List(53f43b03dabfaedce555bf2a, null, null, null, Min Pan, null, null, null, null, null, null, null, null, null, null), List(53f45ee9dabfaee43ecda842, null, null, null, Chris C. N. Chu, null, null, null, null, null, null, null, null, null, null), List(53f42e8cdabfaee1c0a4274e, null, null, null, Hai Zhou, null, null, null, null, null, null, null, null, null, null))",10.1109/ISCAS.2005.1465124,"List(Delay calculation, Timing failure, Monte Carlo method, Sequential logic, Statistical static timing analysis, Shortest path problem, Computer science, Algorithm, Clock skew, Static timing analysis, Statistics)",0-7803-8834-8,null,"List(sequential circuits, statistical distributions, set-up time constraints, register-to-register paths, statistical static timing analysis, integrated circuit modelling, parameter estimation, statistical analysis, circuit model, path delays, deep sub-micron technology, timing, delay distributions, delays, circuit timing, shortest path variations, hold time constraints, integrated circuit yield, process variations, integrated circuit layout, high-performance circuit designs, clock skew, timing yield estimation, 

### 3. Create the new DFs
#### 3.1. Venue DF

In [0]:
# coalescing venue._id and venue.issn to make up for missing ids
_df = _df.withColumn("venue", F.col("venue").withField("_id", F.coalesce(F.col("venue._id"), F.col("venue.issn"))))

# creating the venues df
venues_df = (_df
             .withColumn("has_volume_or_issue", F.when(F.col("volume").isNotNull() | F.col("issue").isNotNull(), True).otherwise(None))
             .select("venue.*", "has_volume_or_issue")
             .filter(F.col("_id").isNotNull())
             .drop("src", "sid", "type").distinct())

# removing the columns from the original df
_df = _df.withColumn("venue_id", F.col("venue._id")).drop("venue")

display(venues_df.limit(DISPLAY_LIMIT))

_id,issn,name,name_d,name_s,online_issn,publisher,raw,raw_zh,t,has_volume_or_issue
0148-9267,0148-9267,null,null,null,null,null,Computer Music Journal,null,J,true
53a72e2020f7420be8c80142,null,null,International Symposium on Circuits and Systems,null,null,null,ISCAS (3),null,null,null
555037b07cea80f95419636b,0036-8733,null,null,null,null,null,ICIG,null,null,true
555037277cea80f954176d25,null,null,null,null,null,null,ISQED,null,null,null
53e181a120f7dfbc07e8bac9,null,null,null,null,null,null,Sexually Transmitted Diseases,null,null,true
53a72dab20f7420be8c6da7e,null,null,null,null,null,null,ICNS,null,null,null
53a72a4920f7420be8bfa51b,null,null,International Conference on Document Analysis and Recognition,null,null,null,ICDAR-1,null,null,null
53a72e9920f7420be8c93fac,null,null,Computer Software and Applications Conference,null,null,null,COMPSAC,null,null,null
53a731f720f7420be8d6ea87,null,null,null,null,null,null,Computer Music Journal,null,null,true
0032-1052,0032-1052,null,null,null,1529-4242,null,PLASTIC AND RECONSTRUCTIVE SURGERY,null,J,true


In [0]:
# combining rows with the same id, but different column values
# taking the first non-null value for the id as the column value

venue_columns = (
    "issn",
    "name",
    "name_d",
    "name_s",
    "raw",
    "raw_zh",
    "online_issn",
    "publisher",
    "t",
    "has_volume_or_issue"
)

venues_df = venues_df.groupBy(F.col("_id")).agg(*(F.first(F.col(col), ignorenulls=True).alias(col) for col in venue_columns))

In [0]:
venues_df = (
    venues_df
    # coalescing the name and raw columns
    .withColumn("raw", F.coalesce(
            F.col("raw"), 
            F.col("raw_zh"),
        ))
    .withColumn("name", F.coalesce(
            F.col("name"), 
            F.col("name_d"),
        ))
    .drop("name_d", "name_s", "raw_zh") 
    # creating the type field
    .withColumn("type",            
               F.when(
                   (
                       F.col("raw").contains("@") | 
                       F.lower(F.col("raw")).contains("workshop") |
                       F.lower(F.col("name")).contains("workshop")
                   ), 
                   "Workshop"
               ).when(
                   (F.col("t") == "J"),
                   "Journal"
               ).when(
                   (
                       (F.col("t") == "C") |
                       F.lower(F.col("raw")).contains("conference") |
                       F.lower(("name")).contains("conference") |
                       F.lower(F.col("raw")).contains("symposium") |
                       F.lower(("name")).contains("symposium") |
                       F.lower(F.col("raw")).contains("proceedings") |
                       F.lower(("name")).contains("proceedings")
                   ),
                   "Conference"
               ).when(
                   (
                       F.lower(F.col("raw")).contains("journal") |
                       F.lower(("name")).contains("journal") |
                       F.col("has_volume_or_issue")
                   ),
                   "Journal"
               ).otherwise(None)
      )
    .drop("t", "has_volume_or_issue")
    # creating the new id
    .withColumn('id', F.monotonically_increasing_id())
)

display(venues_df.limit(DISPLAY_LIMIT))

_id,issn,name,raw,online_issn,publisher,type,id
0001-0782,0001-0782,null,COMMUNICATIONS OF THE ACM,1557-7317,null,Journal,0
0001-2092,0001-2092,null,AORN Journal,null,No longer published by Elsevier,Journal,1
0001-253X,0001-253X,null,ASLIB PROCEEDINGS,1758-3748,null,Journal,2
0001-2815,0001-2815,null,TISSUE ANTIGENS,null,null,Journal,3
0001-4346,0001-4346,null,MATHEMATICAL NOTES,null,null,Journal,4
0001-4966,0001-4966,null,JOURNAL OF THE ACOUSTICAL SOCIETY OF AMERICA,1520-8524,null,Journal,5
0001-5903,0001-5903,null,Acta Informatica,null,null,Journal,6
0001-8708,0001-8708,null,Advances in Mathematics,null,Academic Press,Journal,7
0001-9720,0001-9720,null,AFRICA,null,null,Journal,8
0002-8223,0002-8223,null,Comput. Law Secur. Rev.,null,null,Journal,9


In [0]:
# Join ID to original dataframe (_df) as a long-type column 'Venue'
_df = _df.join(venues_df.select(F.col("_id").alias("venue_id"), F.col("id").alias("new_venue_id")), "venue_id", "leftouter").drop("venue_id").withColumnRenamed("new_venue_id", "venue_id")
display(_df.limit(DISPLAY_LIMIT))

_id,authors,doi,fos,isbn,issue,keywords,lang,n_citation,page_end,page_start,pdf,references,title,url,volume,year,venue_id
53e99784b7602d9701f3e133,"List(List(53f45728dabfaec09f209538, null, null, null, Peijuan Wang, null, null, null, null, null, null, null, null, null, null), List(5601754345cedb3395e59457, null, null, null, Jiahua Zhang, null, null, null, null, null, null, null, null, null, null), List(53f38438dabfae4b34a08928, null, null, null, Donghui Xie, null, null, null, null, null, null, null, null, null, null), List(5601754345cedb3395e5945a, null, null, null, Yanyan Xu, null, null, null, null, null, null, null, null, null, null), List(53f43d25dabfaeecd6995149, null, null, null, Yun Xu, null, null, null, null, null, null, null, null, null, null))",10.1109/IGARSS.2011.6049503,"List(Agronomy, Moisture, Hydrology, Environmental science, Dry weight, Water content, Stomatal conductance, Transpiration, Irrigation, Soil water, Canopy)",null,null,"List(canopy parameters, canopy spectrum, different soil water content control, winter wheat, irrigation, hydrology, radiometry, moisture, indexes, vegetation, indexation, dry weight, soil moisture, water content, indexing terms, spectrum, natural disaster)",en,0,1933,1930,null,null,The relationship between canopy parameters and spectrum of winter wheat under different irrigations in Hebei Province.,List(http://dx.doi.org/10.1109/IGARSS.2011.6049503),null,2011,8021
53e99784b7602d9701f3e151,"List(List(53f46797dabfaeb22f542630, null, null, null, Jairo Rocha, null, null, null, null, null, null, null, null, null, null), List(54328883dabfaeb4c6a8a699, null, null, null, Theo Pavlidis, null, null, null, null, null, null, null, null, null, null))",10.1109/ICDAR.1993.395663,"List(Intelligent character recognition, Pattern recognition, Computer science, Feature (computer vision), Document processing, Handwriting recognition, Optical character recognition, Feature extraction, Feature (machine learning), Artificial intelligence, Intelligent word recognition)",null,null,"List(handwriting recognition, prototypes, image segmentation, computer science, expert systems, knowledge base, pattern recognition, usability, optical character recognition, shape, feature extraction)",en,17,605,602,null,"List(53e99cf5b7602d97025ace63, 557e8a7a6fee0fe990caa63d, 53e9a96cb7602d97032c459a, 53e9b929b7602d9704515791, 557e59ebf6678c77ea222447)",A solution to the problem of touching and broken characters.,List(http://dx.doi.org/10.1109/ICDAR.1993.395663),null,1993,3130
53e99784b7602d9701f3e15d,"List(List(53f43b03dabfaedce555bf2a, null, null, null, Min Pan, null, null, null, null, null, null, null, null, null, null), List(53f45ee9dabfaee43ecda842, null, null, null, Chris C. N. Chu, null, null, null, null, null, null, null, null, null, null), List(53f42e8cdabfaee1c0a4274e, null, null, null, Hai Zhou, null, null, null, null, null, null, null, null, null, null))",10.1109/ISCAS.2005.1465124,"List(Delay calculation, Timing failure, Monte Carlo method, Sequential logic, Statistical static timing analysis, Shortest path problem, Computer science, Algorithm, Clock skew, Static timing analysis, Statistics)",0-7803-8834-8,null,"List(sequential circuits, statistical distributions, set-up time constraints, register-to-register paths, statistical static timing analysis, integrated circuit modelling, parameter estimation, statistical analysis, circuit model, path delays, deep sub-micron technology, timing, delay distributions, delays, circuit timing, shortest path variations, hold time constraints, integrated circuit yield, process variations, integrated circuit layout, high-performance circuit designs, clock skew, timing yield estimation, deterministic static timing analysis, monte carlo simulation, design method, static timing analysis, design methodology, process variation, shortest path, registers, circuit design, circuit analysis)",en,28,null,2461,//static.aminer.org/pdf/PDF/000/423/329/timing_yield_estimation_using_statistical_sta

In [0]:
# dropping the old id
venues_df = venues_df.drop("_id")
display(venues_df.limit(DISPLAY_LIMIT))

issn,name,raw,online_issn,publisher,type,id
0001-0782,null,COMMUNICATIONS OF THE ACM,1557-7317,null,Journal,0
0001-2092,null,AORN Journal,null,No longer published by Elsevier,Journal,1
0001-253X,null,ASLIB PROCEEDINGS,1758-3748,null,Journal,2
0001-2815,null,TISSUE ANTIGENS,null,null,Journal,3
0001-4346,null,MATHEMATICAL NOTES,null,null,Journal,4
0001-4966,null,JOURNAL OF THE ACOUSTICAL SOCIETY OF AMERICA,1520-8524,null,Journal,5
0001-5903,null,Acta Informatica,null,null,Journal,6
0001-8708,null,Advances in Mathematics,null,Academic Press,Journal,7
0001-9720,null,AFRICA,null,null,Journal,8
0002-8223,null,Comput. Law Secur. Rev.,null,null,Journal,9


#### 3.2. Author DF

In [0]:
# Create the Authors DF

df2 = _df.withColumn('auth_expl', F.explode(F.col("authors"))) # explode the authors array
df2 = (df2.withColumn('auth_id', F.col('auth_expl._id')) # separate the authors id and name
          .withColumn('auth_name', F.col('auth_expl.name')))

authors_df = df2.select('auth_id', 'auth_name').distinct() # make the authors df of distinct auth_id and auth_name pairs
# there were only 92 rows where both auth id and name were null.
# altogether, there are 400k unique authors.

authors_df = (authors_df.withColumnRenamed('auth_id', 'ID')
                        .withColumnRenamed('auth_name', 'Name'))

del df2
display(authors_df.limit(DISPLAY_LIMIT))

ID,Name
53f46a22dabfaee0d9c3d5e5,Shuguo Yang
53f431bcdabfaee2a1cb41b5,Sunyoung Ahn
53f45ee9dabfaee43ecda842,Chris C. N. Chu
548a2e3ddabfae9b40134fbc,Harry M. Sneed
53f39e3edabfae4b34aa8c4a,Jungil Park
53f42e8cdabfaee1c0a4274e,Hai Zhou
53f44f6adabfaedf435efcb8,James Jungho Pak
53f42d8bdabfaec22ba1a1e3,M. M. Gore
53f43b03dabfaedce555bf2a,Min Pan
54328883dabfaeb4c6a8a699,Theo Pavlidis


In [0]:
# Generate the authors FK column in the original DF
_df = _df.withColumn('Author_ID', F.col('authors._id'))
display(_df.limit(DISPLAY_LIMIT))

_id,authors,doi,fos,isbn,issue,keywords,lang,n_citation,page_end,page_start,pdf,references,title,url,volume,year,venue_id,Author_ID
53e99784b7602d9701f3e133,"List(List(53f45728dabfaec09f209538, null, null, null, Peijuan Wang, null, null, null, null, null, null, null, null, null, null), List(5601754345cedb3395e59457, null, null, null, Jiahua Zhang, null, null, null, null, null, null, null, null, null, null), List(53f38438dabfae4b34a08928, null, null, null, Donghui Xie, null, null, null, null, null, null, null, null, null, null), List(5601754345cedb3395e5945a, null, null, null, Yanyan Xu, null, null, null, null, null, null, null, null, null, null), List(53f43d25dabfaeecd6995149, null, null, null, Yun Xu, null, null, null, null, null, null, null, null, null, null))",10.1109/IGARSS.2011.6049503,"List(Agronomy, Moisture, Hydrology, Environmental science, Dry weight, Water content, Stomatal conductance, Transpiration, Irrigation, Soil water, Canopy)",null,null,"List(canopy parameters, canopy spectrum, different soil water content control, winter wheat, irrigation, hydrology, radiometry, moisture, indexes, vegetation, indexation, dry weight, soil moisture, water content, indexing terms, spectrum, natural disaster)",en,0,1933,1930,null,null,The relationship between canopy parameters and spectrum of winter wheat under different irrigations in Hebei Province.,List(http://dx.doi.org/10.1109/IGARSS.2011.6049503),null,2011,8021,"List(53f45728dabfaec09f209538, 5601754345cedb3395e59457, 53f38438dabfae4b34a08928, 5601754345cedb3395e5945a, 53f43d25dabfaeecd6995149)"
53e99784b7602d9701f3e151,"List(List(53f46797dabfaeb22f542630, null, null, null, Jairo Rocha, null, null, null, null, null, null, null, null, null, null), List(54328883dabfaeb4c6a8a699, null, null, null, Theo Pavlidis, null, null, null, null, null, null, null, null, null, null))",10.1109/ICDAR.1993.395663,"List(Intelligent character recognition, Pattern recognition, Computer science, Feature (computer vision), Document processing, Handwriting recognition, Optical character recognition, Feature extraction, Feature (machine learning), Artificial intelligence, Intelligent word recognition)",null,null,"List(handwriting recognition, prototypes, image segmentation, computer science, expert systems, knowledge base, pattern recognition, usability, optical character recognition, shape, feature extraction)",en,17,605,602,null,"List(53e99cf5b7602d97025ace63, 557e8a7a6fee0fe990caa63d, 53e9a96cb7602d97032c459a, 53e9b929b7602d9704515791, 557e59ebf6678c77ea222447)",A solution to the problem of touching and broken characters.,List(http://dx.doi.org/10.1109/ICDAR.1993.395663),null,1993,3130,"List(53f46797dabfaeb22f542630, 54328883dabfaeb4c6a8a699)"
53e99784b7602d9701f3e15d,"List(List(53f43b03dabfaedce555bf2a, null, null, null, Min Pan, null, null, null, null, null, null, null, null, null, null), List(53f45ee9dabfaee43ecda842, null, null, null, Chris C. N. Chu, null, null, null, null, null, null, null, null, null, null), List(53f42e8cdabfaee1c0a4274e, null, null, null, Hai Zhou, null, null, null, null, null, null, null, null, null, null))",10.1109/ISCAS.2005.1465124,"List(Delay calculation, Timing failure, Monte Carlo method, Sequential logic, Statistical static timing analysis, Shortest path problem, Computer science, Algorithm, Clock skew, Static timing analysis, Statistics)",0-7803-8834-8,null,"List(sequential circuits, statistical distributions, set-up time constraints, register-to-register paths, statistical static timing analysis, integrated circuit modelling, parameter estimation, statistical analysis, circuit model, path delays, deep sub-micron technology, timing, delay distributions, delays, circuit timing, shortest path variations, hold time constraints, integrated circuit yield, process variations, integrated circuit layout, high-performance circuit designs, clock skew, timing yield estimation, deterministic static timing analysis, monte carlo simulation, design method, static timing analy

#### 3.3. Organization DF

In [0]:
# Create the new df
import re

def getCountry(s):
    if s is None:
        return None
    arr = []
    countries = ["Afghanistan", "Albania", "Algeria", "Andorra", "Angola", "Antigua & Deps", "Argentina", "Armenia", "Australia", "Austria", "Azerbaijan", "Bahamas", "Bahrain", "Bangladesh", "Barbados", "Belarus", "Belgium", "Belize", "Benin", "Bhutan", "Bolivia", "Bosnia", "Botswana", "Brazil", "Brunei", "Bulgaria", "Burkina", "Burundi", "Cambodia", "Cameroon", "Canada", "Cape Verde", "Central African Republic", "Chad", "Chile", "China", "Colombia", "Comoros", "Congo", "Congo Democratic Republic", "Costa Rica", "Croatia", "Cuba", "Cyprus", "Czech Republic", "Denmark", "Djibouti", "Dominica", "Dominican Republic", "East Timor", "Ecuador", "Egypt", "El Salvador", "Equatorial Guinea", "Eritrea", "Estonia", "Ethiopia", "Fiji", "Finland", "France", "Gabon", "Gambia", "Georgia", "Germany", "Ghana", "Greece", "Grenada", "Guatemala", "Guinea", "Guinea-bissau", "Guyana", "Haiti", "Honduras", "Hungary", "Iceland", "India", "Indonesia", "Iran", "Iraq", "Ireland", "Israel", "Italy", "Ivory Coast", "Jamaica", "Japan", "Jordan", "Kazakhstan", "Kenya", "Kiribati", "South Korea", "Kosovo", "Kuwait", "Kyrgyzstan", "Laos", "Latvia", "Lebanon", "Lesotho", "Liberia", "Libya", "Liechtenstein", "Lithuania", "Luxembourg", "Macedonia", "Madagascar", "Malawi", "Malaysia", "Maldives", "Mali", "Malta", "Marshall Islands", "Mauritania", "Mauritius", "Mexico", "Micronesia", "Moldova", "Monaco", "Mongolia", "Montenegro", "Morocco", "Mozambique", "Myanmar", "Burma", "Namibia", "Nauru", "Nepal", "Netherlands", "New Zealand", "Nicaragua", "Niger", "Nigeria", "Norway", "Romania", "Pakistan", "Palau", "Panama", "Papua New Guinea", "Paraguay", "Peru", "Philippines", "Poland", "Portugal", "Qatar", "Oman", "Russia", "Rwanda", "St Kitts & Nevis", "St Lucia", "Saint Vincent & The Grenadines", "Samoa", "San Marino", "Sao Tome & Principe", "Saudi Arabia", "Senegal", "Serbia", "Seychelles", "Sierra Leone", "Singapore", "Slovakia", "Slovenia", "Solomon Islands", "Somalia", "South Africa", "South Sudan", "Spain", "Sri Lanka", "Sudan", "Suriname", "Swaziland", "Sweden", "Switzerland", "Syria", "Taiwan", "Tajikistan", "Tanzania", "Thailand", "Togo", "Tonga", "Trinidad & Tobago", "Tunisia", "Turkey", "Turkmenistan", "Tuvalu", "Uganda", "Ukraine", "United Arab Emirates", "United Kingdom", "United States", "Uruguay", "Uzbekistan", "Vanuatu", "Vatican City", "Venezuela", "Vietnam", "Yemen", "Zambia", "Zimbabwe"]
    state_names = ["alaska", "alabama", "arkansas", "american samoa", "arizona", "california", "colorado", "connecticut", "district ", "of columbia", "delaware", "florida", "georgia", "guam", "hawaii", "iowa", "idaho", "illinois", "indiana", "kansas", "kentucky", "louisiana", "massachusetts", "maryland", "maine", "michigan", "minnesota", "missouri", "mississippi", "montana", "north carolina", "north dakota", "nebraska", "new hampshire", "new jersey", "new mexico", "nevada", "new york", "ohio", "oklahoma", "oregon", "pennsylvania", "puerto rico", "rhode island", "south carolina", "south dakota", "tennessee", "texas", "utah", "virginia", "virgin islands", "vermont", "washington", "wisconsin", "west virginia", "wyoming"]
    states = ['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']
    
    for i in s:
        if i["org"] is None:
            arr.append(None)
            break
        sent = re.sub("[^a-zA-Z -]", "", i["org"])
        x = None
        for j in countries:
            x = re.search(j.lower(), sent.lower())
            if x is not None:
                if j.lower() == 'india':
                    x = re.search('indiana', sent.lower())
                    if x is not None:
                        arr.append("United States")
                elif j.lower() == 'georgia':
                    x = re.search('USA', sent)
                    if x is not None:
                        arr.append("United States")
                else:
                    arr.append(j)
                break
        if x is None:
            x = re.search("USA", sent)
            if x is not None:
                arr.append("United States")
                break
        if x is None:
            x = re.search("UK", sent)
            if x is not None:
                arr.append("United Kingdom")
                break
        if x is None:
            x = re.search("england", sent.lower())
            if x is not None:
                arr.append("United Kingdom")
                break
        if x is None:
            x = re.search("scotland", sent.lower())
            if x is not None:
                arr.append("United Kingdom")
                break
        if x is None:
            x = re.search("wales", sent.lower())
            if x is not None:
                arr.append("United Kingdom")
                break
        if x is None:
            for j in states:
                x = re.search(j, sent)
                if x is not None:
                    arr.append("United States")
                    break
        if x is None:
            for j in state_names:
                x = re.search(j, sent.lower())
                if x is not None:
                    arr.append("United States")
                    break
        break
                    
    if len(arr) > 0:
        return arr[0]
    else:
        return None

getCountryUDF = udf(getCountry)

# Organization (affiliation of the first author)
# ID - authors.orgid
# Name - authors.org
# Country - getCountryUDF(F.arrays_zip("authors.org"))
def organization(df):
    new_df = df.select(F.col("authors.orgid").getItem(0).alias("org_id"),
                       F.col("authors.org").getItem(0).alias("org_name"),
                       (getCountryUDF(F.arrays_zip("authors.org"))).alias("org_country"))
    new_df = new_df.na.drop("all")
    return new_df

org_df = organization(_df)

new_df = org_df.select('org_id', 'org_name', 'org_country').distinct()
new_df = new_df.withColumn('ID', F.monotonically_increasing_id())

In [0]:
# Join ID to original dataframe (_df)

# TODO: make sure that at the end, _df has the column 'Org' with the FK of the organization.
# TODO: Fix the error

df_temp = _df.withColumn("org_id", F.col("authors.orgid").getItem(0)).withColumn("org_name", F.col("authors.org").getItem(0))
df_with_id = df_temp.join(new_df.select('ID', 'org_id', 'org_name'), on=['org_id', 'org_name'])
df_with_id = final_df.drop('org_id', 'org_name')

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-390784252746146> in <module>
      2 df_temp = _df.withColumn("org_id", F.col("authors.orgid").getItem(0)).withColumn("org_name", F.col("authors.org").getItem(0))
      3 df_with_id = df_temp.join(new_df.select('ID', 'org_id', 'org_name'), on=['org_id', 'org_name'])
----> 4 df_with_id = final_df.drop('org_id', 'org_name')

NameError: name 'final_df' is not defined

#### 3.4. DBLP fact table

In [0]:
dblp_df = _df.select('_id','venue_id','Org','Author_ID','references','keywords','fos','title','n_citation','lang','page_start','page_end','doi','isbn','year','volume','issue')

dblp_df = dblp_df.toDF('ID','Venue','Org','Authors','References','Keywords','FOS','Title','NoCitations','Lang','PageStart','PageEnd','DOI','ISBN','Year','Volume','Issue')

In [0]:
display(dblp_df.limit(DISPLAY_LIMIT))

ID,Venue,Authors,References,Keywords,FOS,Title,NoCitations,Lang,PageStart,PageEnd,DOI,ISBN,Year,Volume,Issue
53e99784b7602d9701f3e133,8021,"List(53f45728dabfaec09f209538, 5601754345cedb3395e59457, 53f38438dabfae4b34a08928, 5601754345cedb3395e5945a, 53f43d25dabfaeecd6995149)",null,"List(canopy parameters, canopy spectrum, different soil water content control, winter wheat, irrigation, hydrology, radiometry, moisture, indexes, vegetation, indexation, dry weight, soil moisture, water content, indexing terms, spectrum, natural disaster)","List(Agronomy, Moisture, Hydrology, Environmental science, Dry weight, Water content, Stomatal conductance, Transpiration, Irrigation, Soil water, Canopy)",The relationship between canopy parameters and spectrum of winter wheat under different irrigations in Hebei Province.,0,en,1930,1933,10.1109/IGARSS.2011.6049503,null,2011,null,null
53e99784b7602d9701f3e151,3130,"List(53f46797dabfaeb22f542630, 54328883dabfaeb4c6a8a699)","List(53e99cf5b7602d97025ace63, 557e8a7a6fee0fe990caa63d, 53e9a96cb7602d97032c459a, 53e9b929b7602d9704515791, 557e59ebf6678c77ea222447)","List(handwriting recognition, prototypes, image segmentation, computer science, expert systems, knowledge base, pattern recognition, usability, optical character recognition, shape, feature extraction)","List(Intelligent character recognition, Pattern recognition, Computer science, Feature (computer vision), Document processing, Handwriting recognition, Optical character recognition, Feature extraction, Feature (machine learning), Artificial intelligence, Intelligent word recognition)",A solution to the problem of touching and broken characters.,17,en,602,605,10.1109/ICDAR.1993.395663,null,1993,null,null
53e99784b7602d9701f3e15d,10395,"List(53f43b03dabfaedce555bf2a, 53f45ee9dabfaee43ecda842, 53f42e8cdabfaee1c0a4274e)","List(53e9a8a9b7602d97031f6bb9, 599c7b6b601a182cd27360da, 53e9b443b7602d9703f3e52b, 53e9a6a6b7602d9702fdc57e, 599c7b6a601a182cd2735703, 53e9aad9b7602d970345afea, 5582821f0cf2bf7bae57ac18, 5e8911859fced0a24bb9a2ba, 53e9b002b7602d9703a5c932)","List(sequential circuits, statistical distributions, set-up time constraints, register-to-register paths, statistical static timing analysis, integrated circuit modelling, parameter estimation, statistical analysis, circuit model, path delays, deep sub-micron technology, timing, delay distributions, delays, circuit timing, shortest path variations, hold time constraints, integrated circuit yield, process variations, integrated circuit layout, high-performance circuit designs, clock skew, timing yield estimation, deterministic static timing analysis, monte carlo simulation, design method, static timing analysis, design methodology, process variation, shortest path, registers, circuit design, circuit analysis)","List(Delay calculation, Timing failure, Monte Carlo method, Sequential logic, Statistical static timing analysis, Shortest path problem, Computer science, Algorithm, Clock skew, Static timing analysis, Statistics)",Timing yield estimation using statistical static timing analysis,28,en,2461,null,10.1109/ISCAS.2005.1465124,0-7803-8834-8,2005,null,null
53e99784b7602d9701f3e922,10582,"List(53f39e3edabfae4b34aa8c4a, 53f431bcdabfaee2a1cb41b5, 53f46ac3dabfaeee22a63eab, 53f44f6adabfaedf435efcb8)",null,null,null,International Conference on Nano/Micro Engineered and Molecular Systems,1,en,1054,1057,10.1109/NEMS.2009.5068754,null,2009,null,null
53e99784b7602d9701f3f411,7268,List(548a2e3ddabfae9b40134fbc),"List(53e9adbdb7602d97037be8a2, 53e9bb53b7602d9704792f33, 558aa425e4b0b32fcb37fff4, 558abd44e4b031bae1f9653a, 53e9a326b7602d9702c32229, 53e9b1d7b7602d9703c6ce7c, 558a7de784ae84d265bdee99, 53e9ae17b7602d9703828d13, 53e9aa4fb7602d97033bf9ad)","List(Internet, hypermedia markup languages, information resources, systems re-engineering, COBOL, PL/I, World Wide Web, XML, batch programs, data conversion, e-commerce, eXtensible Markup Language, enterprise application integration, interface reengineering, 

In [0]:
dblp_df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Venue: long (nullable = true)
 |-- Authors: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- References: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Keywords: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- FOS: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Title: string (nullable = true)
 |-- NoCitations: integer (nullable = true)
 |-- Lang: string (nullable = true)
 |-- PageStart: integer (nullable = true)
 |-- PageEnd: integer (nullable = true)
 |-- DOI: string (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Issue: integer (nullable = true)



### 4. Load DFs as Delta tables

In [0]:
# DBLP fact table


In [0]:
# Venue table
venues_df.write.format("delta").mode("overwrite").saveAsTable("venues") 

In [0]:
# Author table


In [0]:
# Organization
